In [65]:
import numpy as np
import os
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from math import sqrt

In [66]:
def extract_data(path,create_test=None):
    X_train=[]
    y_train=[]
    X_test=[]
    y_test=[]

    file=os.listdir(path)

    for entity in file:
        try:
            entityPath=os.listdir(f"{path}/{entity}")
            for i,data in enumerate(entityPath):
                try:
                    current_img=cv2.imread(f"{path}/{entity}/{data}")
                    if current_img is not None:
                        if (create_test is not None and ((len(entityPath)>1) and (i==0))):
                            X_test.append(current_img)
                            y_test.append(entity)
                        else:
                            X_train.append(current_img)
                            y_train.append(entity)
                    else:
                        print(f"{path}/{entity}/{data} is None type")
                except Exception as e:
                    print(f"Erreur avec {path}/{entity}/{data} : {e}")
        except Exception as ex:
            print(f"Erreur avec {path}/{entity} : {ex}")
    
    if create_test is not None:
        return X_train,y_train,X_test,y_test
    else:
        return X_train,y_train

In [67]:
def imgProcess(img,withTorch=None):
    if not isinstance(img,np.ndarray):
        print("convertion de l'img")
        img=cv2.imread(img)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY) #RGB -> Gray -> dim(255,255,3)=255*255*3 val de couleur = 195075 vals -> dim(255,255,1) -> 255*255*1 val de couleur = 65025 vals pour l'exemple du 255*255
    img=cv2.resize(img,(100,100)) #dim -> dim(100,100,1) -> 100*100*1 = 10000 val de couleur
    if withTorch is None:
        img=img.flatten() #2D -> 1D vec de (10000,1)
    else:
        img=img.astype(np.float32)/255.0 #rescale entre 0 et 1
        img=img[np.newaxis,:,:] #ajout d'une dimension pour la couleur
        img=torch.tensor(img).unsqueeze(0) #ajout de la dimension de batch necessaire pour pytorch
    return img

In [68]:
def distEuclid(img1,img2): #p1(a1,a2,a3...,a10000) p2(b1,b2,b3...,b10000) car chaque image a 10000 val de couleur
    #img1 = img1.astype(np.int32) #pour avoir des valeur négatives sinon en uint8 c'est de 0 à 255 (pas negatif)
    #img2 = img2.astype(np.int32)
    diff=img1.astype(np.int32)-img2.astype(np.int32) #opti

    #distance euclidienne entre deux points (en 2D) p1(x1,y1) et p2(x2,y2): d=hypothénuse du triangle rectangle
    #ainsi d²=a²+b² et d=sqrt(a²+b²) avec a=abs(x1-x2) et b=abs(y1-y2) DONC -> d=sqrt((x1-x2)²+(y1-y2)²)
    #en n dim : d=sqrt((a1-b1)²+(a2-b2)²...+(an-bn)²) = sqrt(sum[1;N]((a-b)²))
    #return sqrt(sum((a-b)**2 for a,b in zip(img1,img2))) #zip -> pour deux listes
    return np.sqrt(np.sum(diff**2)) #opti

In [69]:
def knn_predict(X_train,y_train, newImg,k):
    distances=[]
    for i in range(len(X_train)):
        dist=distEuclid(X_train[i],newImg) #on calcule la distance entre la nouvelle image et toutes les autres images
        distances.append((dist,y_train[i])) #on stock la distance avec le label de chaque image à laquelle on a calculé la distance
    
    distances.sort(key=lambda x: x[0]) #on trie la liste de distances dans l'ordre croissant comme ça les derniers termes sont les plus proche de la nouvelle image
    k_voisins=distances[:k] #on prend les k plus proches voisins

    counts={} #dictionnaire pour compter les labels les plus proches et leur occurences
    for _,label in k_voisins: #dans le tableau c'est [distance,label], ici on veut que le label donc _
        counts[label]=counts.get(label,0)+1 #.get(a,b) = prend la val a si existe sinon prend la val b

    return max(counts,key=counts.get) #compare les valeurs mais renvoie le label du dictionnaire

In [70]:
def knn_evaluate(processed_train,y_train,processed_test,y_test,k,n):
    knn_results=[]
    true=0
    if n is None:
        n=len(processed_test)

    print(f"Number of sample set to : {n}")
    
    for i,img in enumerate(processed_test[:n]):
        print(f"Image KNN : {i}")
        knn_results.append(knn_predict(processed_train,y_train,processed_test[i],k))
        if knn_results[i]==y_test[i]:
            true+=1
    print(f"Accuracy (%) : {100*true/n}")

    return knn_results

In [71]:
def KNN(X_train,y_train,X_test,y_test,k=3,n=None):
    processed_train=[]
    for img in X_train:
        processed_train.append(imgProcess(img))

    processed_test=[]
    for img in X_test:
        processed_test.append(imgProcess(img))
    print("Processing done.\n")

    knn_result=knn_evaluate(processed_train,y_train,processed_test,y_test,k,n)
    return knn_result

In [72]:
def imgPadding(img,padding):
    size=img.shape[0]
    if padding is not None:
        size=size+2*padding
        newImg=np.zeros((size,size,3),dtype=np.uint8)
        for x in range(padding,size-padding):
            for y in range(padding,size-padding):
                newImg[x][y]=img[x-padding][y-padding]
    else:
        newImg=img
    return newImg

In [73]:
def max_pooling(img,dim=2,padding=None,stride=1):
    if padding is not None:
        img=imgPadding(img,padding)
    
    newDim=((img.shape[0]-dim)//stride)+1
    newImg=np.zeros([newDim,newDim,3])
    
    for rgb in range(img.shape[2]):
        for x in range(0,(img.shape[0]//dim)*dim,stride):
            for y in range(0,(img.shape[0]//dim)*dim,stride):
                max=0
                for h in range(dim):
                    for l in range(dim):
                        if img[x+l][y+h][rgb]>max:
                            max=img[x+l][y+h][rgb]
                newImg[((x-dim)//stride)+1][((y-dim)//stride)+1][rgb]=max
    
    return newImg

In [74]:
def average_pooling(img,dim=2,padding=None,stride=1):
    if padding is not None:
        img=imgPadding(img,padding)
    
    newDim=((img.shape[0]-dim)//stride)+1
    newImg=np.zeros([newDim,newDim,3])
    
    for rgb in range(img.shape[2]):
        for x in range(0,(img.shape[0]//dim)*dim,stride):
            for y in range(0,(img.shape[0]//dim)*dim,stride):
                somme=0
                for h in range(dim):
                    for l in range(dim):
                        somme+=img[x+l][y+h][rgb]
                newImg[((x-dim)//stride)+1][((y-dim)//stride)+1][rgb]=somme//dim*dim
    
    return newImg

In [75]:
def make_model(nb_classes):
    model=nn.Sequential(
        nn.Conv2d(1,16,kernel_size=3,padding=1), #(1,100,100) -> (16,100,100)
        nn.ReLU(),
        nn.MaxPool2d(2,2), #(16,100,100) -> ((100-2)//2)+1 = 49+1 = 50 donc -> (16,50,50)
        nn.Conv2d(16,32,kernel_size=3,padding=1), #(16,50,50) -> (32,50,50)
        nn.ReLU(),
        nn.MaxPool2d(2,2), #(32,50,50) -> ((50-2)//2)+1 = 25 -> (32,25,25)
        nn.Flatten(), #(32,25,25) -> (32*25*25) = (20000)
        #nn.Dropout(0.3) #pour éviter l'overfitting
        nn.Linear(20000,128),
        nn.ReLU(),
        nn.Linear(128,nb_classes)
    )
    return model

In [87]:
def CNN_train(X_train,y_train,nb_epoch,labels,batch_size,model=None):
    if model is None: #créé un model si on en donne pas
        model=make_model(len(list(set(y_train))))
    
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu") #set le device to the GPU or CPU
    model.to(device) #envoie le model au device

    y_train_tensor=torch.tensor([labels[y] for y in y_train])

    X_train_tensor=[imgProcess(img,withTorch=True) for img in X_train] #applique l'imgprocess en mode torch aux images train
    X_train_tensor=torch.cat(X_train_tensor,dim=0) #stock toutes les images stockées dans X_train_tensor dans un conteneur tensor = créé le batch de taille len(X_train)
    
    dataset=TensorDataset(X_train_tensor,y_train_tensor) #assemble nos data pour créer le dataset tensor
    dataloader=DataLoader(dataset,batch_size=batch_size,shuffle=True) #shuffle -> pour changer l'ordre des pairs img-label pour pas qu'on choppe des batchs uniquement du même label
    #le dataloader va séparer le dataset de taille N en (N//batch_size)+1 si N%batch_size != 0 -> il y aura N//batch_size batchs complets et un batch plus petit avec les data restantes
    #on peut rajouter le param drop_last=True si on veut supprimer le dernier batch -> +stabilité et certaines couches demandent un full batch (BatchNorm par ex)
    #+ le batch_size est élevé + ce sera précis mais + ça consommera

    criterion = torch.nn.CrossEntropyLoss() #fonction de coût/perte à minimiser -> crossentropy utilisée pour la classification multiclasse
    #applique softmax à la prédiction (qui sont des scores) pour transformer les scores en probabilités puis compare les proba à la classe cible et renvoie une erreur.
    #plus la probabilité de la bonne classe est faible, plus la perte sera grande
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #lr = learning rate -> plus c'est petit plus c'est progressif et donc plus il aura besoin d'apprentissage mais plus il sera précis
    #Adam = Adaptive Moment Estimation -> optimiseur pour réduire la perte. Adam adapte le lr localement donc pas besoin de régler le lr très finement. Utilise le SGD (Stochastic Gradient Descent)
    #SGD -> minimisation de la perte pour chaque exemple individuellement là ou le classique GD minimise la perte globale -> SGD plus bruité (moins stable) mais + rapide

    model.train() #active le mode training
    for epoch in range(nb_epoch): # 1 epoque = le moment où le modèle a vu tout le dataset donc une époque = tous les (N//batch_size)+1 batchs
        # plus il y a d'epoch + le model va apprendre et réduire l'erreur mais attention à l'overfitting
        #-> petit dataset(>1000) = 10 to 50 epochs -> moyen dataset (1000 to 100k) = 20 to 100 et grand dataset (>100k) -> 10 to 50
        #SINON -> utiliser early stopping pour être précis qui stop quand la précision sur validation baisse ou n'augmente plus -> à coder soit même
        total_loss=0.0
        correct=0
        total=0

        for img,labels in dataloader:
            img=img.to(device) #envoie les img au device
            labels=labels.to(device) #envoie les labels au device, tout à besoin d'être sur le même device pour être traité

            outputs=model(img) #prédiction des img (forward)
            loss=criterion(outputs,labels) #calcul de la perte avec le criterion (ici crossentropy)

            #rétropropagation (backward)
            optimizer.zero_grad() #remet les gradients à 0
            loss.backward()  #calcul les nouveaux gradients

            optimizer.step() #mise à jour des poids

            #partie statistiques (facultatif mais utile)
            total_loss+=loss.item() #additionne la loss de chaque epoch
            _,predicted=torch.max(outputs,1) #prend le score max de la dim 1 de output qui correspond à la classe et prend uniquement la classe associée à ce score max = les classes prédites
            correct+=(predicted==labels).sum().item() #compte combien de ces preds sont correctes
            total+=labels.size(0) #compte le nb d'img traitées dans cette epoch (labels.size(0)=32 ici vu que c'est le batch_size)
            
        acc=100*correct/total #bonnes prédictions sur l'ensemble de l'epoch en %
        print(f"Epoch : {epoch+1}/{nb_epoch}\n Perte :{total_loss:.4f}\n Précision : {acc:.2f}%")
    
    print("Training terminé.")
    return model

In [88]:
def CNN_evaluate(X_test,y_test,model,labels,batch_size):
        device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        
        model.eval() #active le mode evaluation

        y_test_tensor=torch.tensor([labels[y] for y in y_test])

        X_test_tensor=[imgProcess(img,withTorch=True) for img in X_test]
        X_test_tensor=torch.cat(X_test_tensor,dim=0)
    
        test_dataset=TensorDataset(X_test_tensor,y_test_tensor)
        testloader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)

        correct=0
        total=0

        with torch.no_grad(): #pas de backward (juste forward = pred)
            for img,labels in testloader:
                  img=img.to(device)
                  labels=labels.to(device)

                  outputs=model(img)
                  _,pred=torch.max(outputs,1)

                  correct+=(pred==labels).sum().item()
                  total+=labels.size(0)

        print(f"Précision sur le test : {correct/total *100:.2f}%")

In [78]:
def CNN_inf(img_path,model,labels):
    device=torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    img=imgProcess(img_path,withTorch=True).to(device)

    with torch.no_grad():
        output=model(img)
        _,pred=torch.max(output,1)
    
    return labels[pred.item()]

In [85]:
#X_train,y_train,X_test,y_test=extract_data("Faces","yes")
X_train,y_train=extract_data("nous/Train")
X_test,y_test=extract_data("nous/Test")

Invalid SOS parameters for sequential JPEG


In [91]:
labels={label:i for i,label in enumerate(sorted(set(y_train)))} #list(set(y_train)) sort les val uniques en list et sorted les range dans l'ordre alphabétique 
model=CNN_train(X_train=X_train,y_train=y_train,nb_epoch=5,labels=labels,batch_size=3)

Epoch : 1/5
 Perte :5.8773
 Précision : 65.00%
Epoch : 2/5
 Perte :3.9596
 Précision : 85.00%
Epoch : 3/5
 Perte :2.6721
 Précision : 85.00%
Epoch : 4/5
 Perte :1.8927
 Précision : 95.00%
Epoch : 5/5
 Perte :0.4024
 Précision : 100.00%
Training terminé.


In [92]:
CNN_evaluate(X_test=X_test,y_test=y_test,model=model,labels=labels,batch_size=3)

Précision sur le test : 86.67%


In [95]:
id2label = {v: k for k, v in labels.items()}
print(CNN_inf("test.jpg",model,id2label))

convertion de l'img
Augustin
